In [265]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import string

import spacy
import nltk
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import scipy.stats as st

#import utils

[nltk_data] Downloading package stopwords to /Users/Sanna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# download german fastText Embeddings
import wget
import gzip
import os
import os.path

# downloading the .vec.gz-files (fasText, facebook)
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz'
target_path = '/Users/Sanna/data/cc.de.300.vec.gz'
if not os.path.isfile(target_path):
    print('downloading...')
    wget.download(url, out=target_path)
    print('done')
else:
    print('file already exists')

downloading...
done


In [11]:
def load_vectors(path=None, limit=None):
    print('loading embeddings ...')
    f = gzip.open(path, 'rb')
    n, d = map(int, f.readline().split())
    data = {}
    counter = 0
    for line in f:
        line = line.decode()
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(tokens[1:], dtype=np.float32)
        if limit:
            if counter >= limit:
                break
            else:
                counter += 1
    f.close()
    print('done')
    return data

In [38]:
embs = load_vectors('/Users/Sanna/data/cc.de.300.vec.gz', limit = 100000)

loading embeddings ...
done


In [47]:
# exploring the embeddings to see what the preprocessing should be like?
print(embs.get('Garten').shape)
print(embs['das'][:5])
print(embs['Das'][:5])
print(embs['ging'][:5])
print(embs['gehst'][:5])
print(embs['gegangen'][:5])
print(embs[','][:5])

(300,)
[-0.016   0.0168  0.11    0.0636  0.0124]
[-0.0561  0.0101  0.0784  0.0929  0.0192]
[-0.0017  0.0986  0.1736  0.0391  0.0817]
[ 0.017  -0.0775  0.012   0.0453 -0.1227]
[0.0263 0.0028 0.0664 0.0028 0.0067]
[-0.0354 -0.0088 -0.018   0.016  -0.0152]


In [40]:
ROOT = Path('/Volumes/INWT/Daten_NLP/') # encrypted folder!
DATA = ROOT / '200707_aachener_zeitung_modified.csv' # text is already minimal preprocessed

In [76]:
df = pd.read_csv(DATA, index_col = 0)
df = df.fillna('') # replacing Nan with emtpy string
df.head()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,titelH3,wordcount,category,city,text_preprocessed,avgTimeOnPage/wordcount,nr_tokens,mean_token_length,nr_tokens_teaser,nr_tokens_titelH1
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,21,7,12,7,1012,,112.444444,42.857143,33.333333,33.333333,...,,769,vm,München/Stuttgart,Frische Luft und Bewegung: Diese Kombination r...,0.146222,796,5.359296,29,9
48620381,19,6,11,5,1484,,185.500000,42.105263,31.578947,26.315789,...,,441,vm,Berlin/Frankfurt/Main,"Der Wecker klingelt, aufstehen! Doch gerade im...",0.420635,452,5.938053,33,8
48622639,2,2,2,2,0,,0.000000,0.000000,100.000000,100.000000,...,,390,vm,Berlin,Eltern auf der Suche nach einem guten Babyphon...,0.000000,396,5.848485,30,7
48623085,32,9,20,9,974,,81.166667,37.500000,28.125000,28.125000,...,,345,vm,Berlin,Spülmaschinentabs sollen kleine Alleskönner se...,0.235266,367,5.594005,30,7
48623259,24,2,7,2,3797,,223.352941,70.833333,8.333333,8.333333,...,,182,vm,Berlin,Make-up hat heutzutage einen Zweck: Es soll da...,1.227214,183,5.622951,22,8


In [263]:
class Preprocessor():
    def __init__(self, delete_stopwords=False, lemmatize=False, delete_punctuation=False):
        self.nlp = spacy.load("de_core_news_sm", disable=['parser', 'ner'])
        #self.nlp = spacy.load("de_core_news_md", disable=['parser', 'ner'])
        self.delete_stopwords = delete_stopwords
        self.lemmatize = lemmatize
        self.delete_punctuation = delete_punctuation
        self.stopwords = nltk.corpus.stopwords.words('german')

    def __call__(self, doc):
        rt = []
        doc = self.nlp(doc)
        
        if self.lemmatize==True:
            for token in doc:
                rt.append(token.lemma_.lower())
        else:
            for token in doc:
                rt.append(token.text)

        if self.delete_punctuation==True:
            rt = [ t for t in rt if t not in string.punctuation ]
        
        if self.delete_stopwords==True:
            if self.lemmatize == True:
                self.stopwords = [ self.nlp(s)[0].lemma_ for s in self.stopwords ]
            rt = [ t for t in rt if t not in self.stopwords ]

        return " ".join(rt)

In [264]:
s = "Das hier sind verschiedene Wörter, um alle Dinge zu testen und anzuschauen."
pr1 = Preprocessor()
print(pr1(s))
pr2 = Preprocessor(delete_stopwords=True)
print(pr2(s))
pr3 = Preprocessor(delete_punctuation=True)
print(pr3(s))
pr4 = Preprocessor(lemmatize=True, delete_stopwords=True, delete_punctuation=True)
print(pr4(s))

Das hier sind verschiedene Wörter , um alle Dinge zu testen und anzuschauen .
Das verschiedene Wörter , Dinge testen anzuschauen .
Das hier sind verschiedene Wörter um alle Dinge zu testen und anzuschauen
verschieden wort ding testen anschauen


In [226]:
def get_averaged_embs(text, preprocessor, embs):
    vector = np.zeros(300)
    text_preprocessed = preprocessor(text)
    tokens = text_preprocessed.split() # preprocessor returns string with " " as seperator, so needs to be split up
    counter = 0
    for t in tokens:
        #print(t)
        if t in embs:
            vector += embs.get(t)
            counter +=1
    #print(counter)
    if counter !=0:
        vector = vector/counter
    return vector

In [235]:
prepr = Preprocessor(lemmatize=True, delete_stopwords=True)

In [236]:
example = get_averaged_embs("Nicht jeder geht gerne ins Schwimmbad, nur weil die Sonne scheint.",
                            preprocessor = prepr,
                            embs = embs)
example.shape

(300,)

In [237]:
#creating train, dev, test
RANDOM_SEED = 123
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_dev, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED, shuffle=True)
print(df_train.shape, df_dev.shape, df_test.shape)

(712, 35) (89, 35) (90, 35)


In [238]:
## features: averaged embeddings of the text (or teaser, title)

In [239]:
# define text base features: text? text_preprocessed? titelH1?

#feature = 'text_preprocessed'
#feature = 'titelH1'
feature = 'teaser' # note: not all have 'teaser' but I replaced Nan wih empty string ""

X_train = np.array([ get_averaged_embs(text, preprocessor=prepr, embs=embs) for text in df_train[feature] ])
X_dev = np.array([ get_averaged_embs(text, preprocessor=prepr, embs=embs) for text in df_dev[feature] ])
X_test = np.array([ get_averaged_embs(text, preprocessor=prepr, embs=embs) for text in df_test[feature] ])

In [240]:
X_train.shape, X_dev.shape, X_test.shape

((712, 300), (89, 300), (90, 300))

In [241]:
# define the target labels
#target = 'timeOnPage'
target =  'pageviews'
#target = 'avgTimeOnPage'
#target = 'stickiness'

y_train = np.array(df_train[target])
y_dev = np.array(df_dev[target])
y_test = np.array(df_test[target])

In [242]:
y_train.shape, y_dev.shape, y_test.shape

((712,), (89,), (90,))

In [243]:
from sklearn.linear_model import Ridge

In [244]:
# choose model ???
model = Ridge()

In [245]:
model.fit(X_train, y_train)

Ridge()

In [246]:
# predict for dev set
pred_dev = model.predict(X_dev)

In [247]:
# postprocessing: replace negative values with 0 (better way? can I give that hint to the model?)
pred_dev[pred_dev < 0] = 0
pred_dev

array([80.34301044, 67.02367867, 32.34249982, 37.97274214, 45.17958433,
       53.46087758,  7.78624909,  0.        , 37.9308578 , 56.85670713,
       33.87112423, 67.61228435, 46.98393575, 27.64920572, 50.2584279 ,
       34.80664752,  6.94935277, 40.22879557, 47.68284705, 54.03364662,
       67.2053647 , 25.35241115, 13.97489079, 13.15936223, 56.52384708,
       34.53634827, 35.75397949,  5.63514561, 33.62525135, 35.16240142,
        5.63514561, 24.69818036, 58.59300743, 27.90721546, 16.49494915,
       36.78567524, 53.0406133 , 36.71100224, 32.87803735, 49.70987304,
       65.21730411, 43.38232696, 57.41243046, 28.64371221,  3.66888605,
       24.27277587, 35.84681489, 47.31376929, 14.08634601, 41.53865671,
       28.36237667, 27.62774779, 44.55897099, 54.36404405, 47.37465727,
        5.63514561, 43.48386224, 12.28266352, 37.78142383, 50.96623779,
       50.83391154, 71.31425673, 34.95882248, 27.63999398, 39.58764615,
       41.74530102, 16.93273127, 47.30209078,  5.63514561, 57.17

In [248]:
np.array(y_dev)

array([3047,   33,   20,   64,    1,   12,    2,   22,    3,    6,    1,
         19,    3,  262,    9,    4,   24,  100,   27,  124,    3,    8,
         14,    6,   39,   14,   16,   18,    9,    8,    8,    9,   16,
         32,   19,    8,   20,    3,   18,   11,   18,    9,    9,   24,
         14,    6,    5,  115,    4,   14,    9,    7,   16,   17,    6,
          8,   19,   39,   17,   29,   24,   13,    1,    8,   10,    3,
          7,   14,    9,    7,    7,    2,    7,    4,   12,    5,   12,
          7,  173,   15,    3,   18,   29,   21,    1,   10,    2,    9,
         18])

In [249]:
st.pearsonr(pred_dev, y_dev)

(0.24747229191073583, 0.019380022351068203)